In [1]:
import numpy as np
from scipy.optimize import minimize

CNOT_UNITARY = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
ISWAP_UNITARY = np.array([[1, 0, 0, 0], [0, 0, -1j, 0], [0, -1j, 0, 0], [0, 0, 0, 1]])

---
## Define Native Two-Qubit Gates

In [2]:
class NativeTwoQubitGate(object):
    num_params = -1

    def get_unitary(self, params=[]):
        assert len(params) == self.num_params
        assert self.num_params != -1, 'Subclass should set num_params'
        assert self.num_params in [0, 1], 'Currently doesn\'t handle multi-parameter native gates'

        return self._get_unitary(params)
    
    def _get_unitary(self, params=[]):
        raise NotImplemented('Subclass should implement')
        
    def __str__(self):
        return self.__class__.__name__

    
class NativeCNOT(NativeTwoQubitGate):
    num_params = 0
    
    def _get_unitary(self, params=[]):
        return CNOT_UNITARY


class NativeiSWAP(NativeTwoQubitGate):
    num_params = 0
    
    def _get_unitary(self, params=[]):
        return ISWAP_UNITARY


class NativeCR(NativeTwoQubitGate):
    num_params = 1
    
    def _get_unitary(self, params=[]):
        theta = params[0]
        return np.array([
            [np.cos(theta/2), -1j * np.sin(theta/2), 0, 0],
            [-1j * np.sin(theta/2), np.cos(theta/2), 0, 0],
            [0, 0, np.cos(theta/2), 1j * np.sin(theta/2)],
            [0, 0, 1j * np.sin(theta/2), np.cos(theta/2)]])


class NativeParametrizediSWAP(NativeTwoQubitGate):
    num_params = 1
    
    def _get_unitary(self, params=[]):
        theta = params[0]
        return np.array([
            [1, 0, 0, 0],
            [0, np.cos(theta), -1j*np.sin(theta), 0],
            [0, -1j*np.sin(theta), np.cos(theta), 0],
            [0, 0, 0, 1]])

    
class NativeBSWAP(NativeTwoQubitGate):
    pass

class NativeMAP(NativeTwoQubitGate):
    pass

class NativeRIP(NativeTwoQubitGate):
    pass

class NativeMS(NativeTwoQubitGate):
    pass

## Define Target Two-Qubit Operations

In [3]:
class TargetTwoQubitOperation(object):
    def get_unitaries(self, params=None):
        raise NotImplemented('Subclass should implement')
        
    def __str__(self):
        return self.__class__.__name__


class TargetCNOT(TargetTwoQubitOperation):
    def get_unitaries(self):
        return [CNOT_UNITARY]


class TargetSWAP(TargetTwoQubitOperation):
    def get_unitaries(self):
        return [np.array([[1, 0, 0, 0],
                          [0, 0, 1, 0],
                          [0, 1, 0 ,0],
                          [0, 0, 0, 1]])]

    
class TargetZZInteraction(TargetTwoQubitOperation):
    def get_unitaries(self, params=None):
        if params is None:
            params = [np.random.random() for _ in range(10)] 
        return [np.diag([1, np.exp(1j * param), np.exp(1j * param), 1]) for param in params]

## Define Minimization Problem

In [6]:
def get_best_decomposition(target_two_qubit_operation, native_two_qubit_gate):
    for k in [1, 2, 3]:
        if has_k_step_decomposition(target_two_qubit_operation, native_two_qubit_gate, k):
            return k
    assert False, 'did not find any 3-step decomposition'
            

def has_k_step_decomposition(target_two_qubit_operation, native_two_qubit_gate, k):
    for target_unitary in target_two_qubit_operation.get_unitaries():
        num_failures = 0
        while True:
            result = k_step_minimization(target_unitary, native_two_qubit_gate, k)
            if result.fun > -0.99:
                num_failures += 1  # retry up to 10 times before giving up
                if num_failures >= 10:
                    return False
            else:
                break
    return True


def k_step_minimization(target_unitary, native_two_qubit_gate, k):
    # there are k+1 single qubit layers and k two qubit layers
    # yielding a total of 2(k+1) single qubit gates and k two qubit gates
    # each single qubit gate has 3 parameters, so they contribute to 6k + 6 parameters
    num_params = (6 * k + 6) + k * (native_two_qubit_gate.num_params)
    params = [np.random.random() for _ in range(num_params)]

    return minimize(get_fidelity, params,
                    args=(target_unitary, native_two_qubit_gate, k),
                    method='Nelder-Mead')


def get_fidelity(params, target_unitary, native_two_qubit_gate, k):
    params = list(params)  # so that we can call .pop()
    single_qubit_u = []  # list of 2(k+1) single qubit gates
    for i in range(2*(k + 1)):
        a, b, c = params[3*i], params[3*i + 1], params[3*i + 2]  # generic single-qubit unitary has 3 params
        single_qubit_u.append(np.array([[np.exp(1j * a) * np.cos(b), np.exp(1j * c) * np.sin(b)],
                                        [-np.exp(-1j * c) * np.sin(b), np.exp(-1j * a) * np.cos(b)]]))

    two_qubit_u = []  # list of k two-qubit gates
    if native_two_qubit_gate.num_params == 0:
        for _ in range(k):
            two_qubit_u.append(native_two_qubit_gate.get_unitary())
    else:
        for i in range(k):
            two_qubit_u.append(native_two_qubit_gate.get_unitary([params[-i]]))

    actual_unitary = np.eye(4)
    for i in range(k):
        actual_unitary = actual_unitary @ np.kron(single_qubit_u.pop(), single_qubit_u.pop())
        actual_unitary = actual_unitary @ two_qubit_u.pop()
    actual_unitary = actual_unitary @ np.kron(single_qubit_u.pop(), single_qubit_u.pop())

    fidelity = np.abs(np.trace(actual_unitary @ np.linalg.inv(target_unitary))) / 4
    return -fidelity  # return negative, since we will minimize

---
## Results

In [7]:
native_two_qubit_gates = [NativeCNOT(), NativeiSWAP(), NativeCR(), NativeParametrizediSWAP()]
target_two_qubit_operations = [TargetCNOT(), TargetSWAP(), TargetZZInteraction()]

In [8]:
results = {}
for target_two_qubit_operation in target_two_qubit_operations:
    for native_two_qubit_gate in native_two_qubit_gates:
        k = get_best_decomposition(target_two_qubit_operation, native_two_qubit_gate)
        results[(target_two_qubit_operation, native_two_qubit_gate)] = k

In [9]:
row_format ="{:>20}" * (len(native_two_qubit_gates) + 1)
print(row_format.format("", *map(lambda s: str(s)[6:], native_two_qubit_gates)))
for target_two_qubit_operation in target_two_qubit_operations:
    row = [results[target_two_qubit_operation, native_two_qubit_gate] for native_two_qubit_gate in native_two_qubit_gates]
    print(row_format.format(str(target_two_qubit_operation)[6:], *row))

                                    CNOT               iSWAP                  CR   ParametrizediSWAP
                CNOT                   1                   2                   1                   2
                SWAP                   3                   3                   3                   3
       ZZInteraction                   2                   2                   1                   2
